<a href="https://colab.research.google.com/github/jorgeveamurguia/prueba/blob/master/Patterns_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
from google.colab.patches import cv2_imshow
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:

!ls "/content/gdrive/MyDrive/ZEU/100-OS2-NoOk-Mota-f21a.jpg"
#pic = cv2.imread('/content/gdrive/drive/u/0/my-drive/photo-1509909756405-be0199881695.jpeg')
#pic = cv2.imread('/content/drivedrive/drive/u/0/my-drive/photo-1509909756405-be0199881695.jpeg')
pic = cv2.imread('/content/gdrive/MyDrive/ZEU/100-OS2-NoOk-Mota-f21a.jpg')

In [ ]:

cv2_imshow(pic)
cv2.waitKey(0)

In [ ]:
import numpy as np
import cv2

In [ ]:
src = cv2.imread("/content/gdrive/MyDrive/ZEU/100-OS2-NoOk-Mota-f21a.jpg")
Temp = cv2.imread("/content/gdrive/MyDrive/ZEU/marcador-Blanco-25.jpg")

In [ ]:
src = cv2.cvtColor(src, cv2.COLOR_RGB2GRAY)
temp = cv2.cvtColor(Temp, cv2.COLOR_RGB2GRAY)

In [ ]:
height, width =src.shape
height, width

In [ ]:
H, W = temp.shape
H, W

In [ ]:
methods = [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR,
            cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]

In [ ]:
for method in methods:
  src2 = src.copy()

  result = cv2.matchTemplate(src2, temp, method)
  min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
  print(min_loc, max_loc)
  if method in [cv2.TM_SQDIFF,cv2.TM_CCORR]:
    location = min_loc
  else:
    location = max_loc
  bottom_right = (location[0] + W, location[1] + H)
  cv2.rectangle(src2, location,bottom_right, 255, 5)
  cv2_imshow(src2)
  cv2.waitKey(0)
  cv2.destroyAllWindows()

In [ ]:
def intersection(a,b):
  x = max(a[0], b[0])
  y = max(a[1], b[1])
  w = min(a[2], b[2]) - x
  h = min(a[3], b[3]) - y
  #print(f"{a} {b}")
  if w<0 or h<0: return (0,0,0,0)
  return (x, y, x + w, y + h)

src4 = src.copy()

result = cv2.matchTemplate(src4, temp, cv2.TM_CCOEFF_NORMED)
threshold = 0.39

loc = np.where( result >= threshold) # filter the results
mark1 = H , W,  height*2, width*2
mark2 = H , W,  height*2, width*2
mark3 = H , W,  height*2, width*2
for pt in zip(*loc[::-1]): #pt marks the location of the match
    #print(pt[0] + W, pt[1] + H)
    a = pt[0] , pt[1], pt[0] + W, pt[1] + H

    test1=intersection(a, mark1 )
    test2=intersection(a, mark2 )
    test3=intersection(a, mark3 )
    if ( (0,0,0,0) != test1):
      aux1 = test1
    elif ( (0,0,0,0) != test2 ):
      aux2 = test2
    elif ( (0,0,0,0) != test3):
      aux3 = test3
    #cv2.rectangle(src4, pt, (pt[0] + W, pt[1] + H), (0,0,255), 2)

cv2.rectangle(src4, (aux1[0], aux1[1] ), (aux1[2], aux1[3]), (255,0,255), 2)
cv2.rectangle(src4, (aux2[0], aux2[1] ), (aux2[2], aux2[3]), (0,255,255), 2)
cv2.rectangle(src4, (aux3[0], aux3[1] ), (aux3[2], aux3[3]), (255,255,0), 2)

cv2_imshow(src4)
cv2.waitKey(0)
cv2.destroyAllWindows()


In [ ]:
src5 = src.copy()
edges = cv2.Canny(src, 50, 150, apertureSize = 3)
lines = cv2.HoughLinesP(edges, 1, np.pi/180, 100, minLineLength=100, maxLineGap=10)
#https://newbedev.com/cropping-concave-polygon-from-image-using-opencv-python
def inside(rect, line):
  print (f"{rect} {line} {rect[0] < line [0] and rect[1] > line [1] and rect[2] > line [0] and rect[3] < line [1] }" )

  return (rect[0] < line [0] and rect[1] > line [1] and rect[2] > line [0] and rect[3] < line [1] ) #and (rect[0] < line [2] and rect[1] > line [3] and rect[2] > line [2] and rect[3] < line [3]  )


for line in lines:
    x1, y1, x2, y2 = line[0]
    if inside (aux1 , line[0] ) or inside (aux2 , line[0] ) or  inside (aux3 , line[0] ):
      cv2.line(src5, (x1,y1), (x2,y2), (0,255,0), 5, cv2.LINE_AA)

cv2_imshow(src5)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [ ]:


src3 = src.copy()

result = cv2.matchTemplate(src3, temp, cv2.TM_CCOEFF_NORMED)


#get all the matches:
result2 = np.reshape(result, result.shape[0]*result.shape[1])
sort = np.argsort(result2)
(y1, x1) = np.unravel_index(sort[0], result.shape) # corner match
(y2, x2) = np.unravel_index(sort[1], result.shape) # corner best match
(y3, x3) = np.unravel_index(sort[2], result.shape) # corner best match


location =  y1 ,x1
bottom_right = (location[0] + W, location[1] + H)
cv2.rectangle(src3, location,bottom_right, 255, 5)

location =  y2 ,x2
bottom_right = (location[0] + W, location[1] + H)
cv2.rectangle(src3, location,bottom_right, 255, 5)

location =  y3 ,x3
bottom_right = (location[0] + W, location[1] + H)
cv2.rectangle(src3, location,bottom_right, 255, 5)

cv2_imshow(src3)
cv2.waitKey(0)
cv2.destroyAllWindows()
